In [2]:
import pandas as pd
from sqlalchemy import create_engine
import statistics as st

In [3]:
engine = create_engine(
    'mssql+pyodbc://'
    '@./AdventureWorksDW2019?'  # username:pwd@server:port/database
    'driver=ODBC+Driver+17+for+SQL+Server'
)

## mean

### query
- mean, median, qunatile. Calculate for each year:
```
SELECT SalesAmount
      ,OrderDate
      ,YEAR(OrderDate)
  FROM [AdventureWorksDW2019].[dbo].[FactInternetSales]
```


In [13]:
df1 = pd.read_sql(
    'SELECT SalesAmount\
      ,OrderDate\
      ,YEAR(OrderDate) year\
  FROM [AdventureWorksDW2019].[dbo].[FactInternetSales]',
    engine,
    coerce_float=True)
df1

,SalesAmount,OrderDate,year
0,3578.2700,2010-12-29,2010
1,3399.9900,2010-12-29,2010
2,3399.9900,2010-12-29,2010
3,699.0982,2010-12-29,2010
4,3399.9900,2010-12-29,2010
...,...,...,...
60393,21.9800,2014-01-28,2014
60394,8.9900,2014-01-28,2014
60395,21.9800,2014-01-28,2014
60396,159.0000,2014-01-28,2014


In [15]:
years = df1.year.unique()
means = {}
for year in years:
    current_year_sales = df1[df1.year==year].SalesAmount
    current_mean = st.median(current_year_sales)
    means[year]= current_mean
means

{2010: 3489.13, 2011: 3578.27, 2012: 2071.4196, 2013: 24.99, 2014: 21.98}

In [43]:
df_2010=df[df['year']==2010]
min(df_2010['SalesAmount']),st.mean(df_2010['SalesAmount']),max(df_2010['SalesAmount'])


(699.0982, 3101.5026, 3578.27)

## quantiles

In [7]:
dec = st.quantiles(df1.SalesAmount, n = 10)
dec

[4.99,
 4.99,
 8.99,
 21.98,
 29.99,
 34.989999999999995,
 53.989999999999995,
 769.49,
 2294.9899999999993]

## spread

### query
```
SELECT f.SalesTerritoryKey,
        st.SalesTerritoryKey,
        st.SalesTerritoryCountry,
        f.SalesAmount
  FROM [AdventureWorksDW2019].[dbo].[FactInternetSales] as f
  JOIN DimSalesTerritory as st ON f.SalesTerritoryKey = st.SalesTerritoryKey
```
calculate per country

In [16]:
df2 = pd.read_sql(
    'SELECT st.SalesTerritoryCountry,\
        f.SalesAmount\
  FROM [AdventureWorksDW2019].[dbo].[FactInternetSales] as f\
  JOIN DimSalesTerritory as st ON f.SalesTerritoryKey = st.SalesTerritoryKey',
    engine,
    coerce_float=True)
df2

,SalesTerritoryCountry,SalesAmount
0,Canada,3578.2700
1,France,3399.9900
2,United States,3399.9900
3,United States,699.0982
4,Australia,3399.9900
...,...,...
60393,Canada,21.9800
60394,Canada,8.9900
60395,Canada,21.9800
60396,Canada,159.0000


In [33]:
#st.variance(df2[df2.SalesTerritoryCountry=='Australia'].SalesAmount)
a = df2[df2.SalesTerritoryCountry=='Australia'].SalesAmount

import numpy as np 
print(np.std(a))

1074.0841787567435


In [32]:
countries = df2.SalesTerritoryCountry.unique()
var_country = {}
for country in countries:
    current_country_sales = df2[df2.SalesTerritoryCountry==country].SalesAmount
    current_var = st.variance(current_country_sales)
    current_std = st.stdev(current_country_sales)
    var_country[country]= (current_var, current_std)
var_country

{'Canada': (522192.34124361194, 722.6287713920696),
 'France': (787371.4512934005, 887.3395355180567),
 'United States': (798403.5634533941, 893.5343101713521),
 'Australia': (1153743.2781531992, 1074.1244239626988),
 'United Kingdom': (810769.4795483286, 900.4273871603021),
 'Germany': (843037.8647697782, 918.1709343960841)}

## correlation

### query
```

  SELECT --f.ProductKey, 
        AVG(TotalProductCost),
        --dp.EnglishProductName,
        dpsc.EnglishProductSubcategoryName,
        dpc.EnglishProductCategoryName,
        YEAR(f.OrderDate)
        --dp.EnglishProductName
  FROM [AdventureWorksDW2019].[dbo].[FactInternetSales] as f
  JOIN DimProduct as dp ON dp.ProductKey = f.ProductKey
  JOIN DimProductSubcategory as dpsc ON dpsc.ProductSubcategoryKey = dp.ProductSubcategoryKey
  JOIN DimProductCategory as dpc ON dpc.ProductCategoryKey = dpsc.ProductCategoryKey
  WHERE dpc.EnglishProductCategoryName IN ('Clothing',  'Accessories') AND  YEAR(f.OrderDate) = 2013 AND  MONTH(f.OrderDate) IN (2,3,4) AND dpsc.EnglishProductSubcategoryName NOT IN ('Helmets','Bike Racks') # the two sequences must be of equal length
  GROUP BY dpc.EnglishProductCategoryName,  YEAR(f.OrderDate), dpsc.EnglishProductSubcategoryName
  ORDER BY YEAR(f.OrderDate) DESC, dpc.EnglishProductCategoryName ASC
```

In [6]:
df3 = pd.read_sql(
    'SELECT AVG(TotalProductCost) AvgTotalProducCost,\
        dpsc.EnglishProductSubcategoryName,\
        dpc.EnglishProductCategoryName,\
        YEAR(f.OrderDate)\
  FROM [AdventureWorksDW2019].[dbo].[FactInternetSales] as f\
  JOIN DimProduct as dp ON dp.ProductKey = f.ProductKey\
  JOIN DimProductSubcategory as dpsc ON dpsc.ProductSubcategoryKey = dp.ProductSubcategoryKey\
  JOIN DimProductCategory as dpc ON dpc.ProductCategoryKey = dpsc.ProductCategoryKey\
  WHERE dpc.EnglishProductCategoryName IN (\'Clothing\',  \'Accessories\') \
    AND YEAR(f.OrderDate) = 2013 \
    AND MONTH(f.OrderDate) IN (2,3,4) \
    AND dpsc.EnglishProductSubcategoryName NOT IN (\'Helmets\',\'Bike Racks\') \
  GROUP BY dpc.EnglishProductCategoryName, YEAR(f.OrderDate), dpsc.EnglishProductSubcategoryName\
  ORDER BY YEAR(f.OrderDate) DESC, dpc.EnglishProductCategoryName ASC',
    engine,
    coerce_float=True)
df3

,AvgTotalProducCost,EnglishProductSubcategoryName,EnglishProductCategoryName,
0,8.2205,Fenders,Accessories,2013
1,5.2414,Tires and Tubes,Accessories,2013
2,20.5663,Hydration Packs,Accessories,2013
3,59.4660,Bike Stands,Accessories,2013
4,2.9733,Cleaners,Accessories,2013
5,2.6235,Bottles and Cages,Accessories,2013
6,23.7490,Vests,Clothing,2013
7,3.3623,Socks,Clothing,2013
8,40.0101,Jerseys,Clothing,2013
9,9.1593,Gloves,Clothing,2013


In [7]:
df_acc = df3[df3.EnglishProductCategoryName == 'Accessories'].AvgTotalProducCost
df_clothing = df3[df3.EnglishProductCategoryName == 'Clothing'].AvgTotalProducCost

st.correlation(df_acc, df_clothing)

-0.08772097464883709